In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from glob import glob
from preprocessing import *

from tqdm import notebook

In [3]:
data_dir = './dataset/'
data_files = glob(data_dir + 'Totaldata*.csv')
new_data_dir = './revised_raw_dataset/'
new_data_files = glob(new_data_dir + 'Totaldata*.csv')
print(len(data_files), len(new_data_files))
print(data_files[0])
print(new_data_files[0])

2492 1145
./dataset\Totaldata_000020_ref_IXIC.csv
./revised_raw_dataset\Totaldata_000020_ref_IXIC.csv


In [4]:
os.path.dirname(data_files[0]), os.path.dirname(new_data_files[0])

('./dataset', './revised_raw_dataset')

In [7]:
# sample concatenation
df1 = load_csv_data(data_files[0])
df2 = load_csv_data(new_data_files[0])
df = pd.concat([df1, df2])
idx = df.index.unique().tolist()
df.drop_duplicates(inplace=True)
print(len(df1), len(df2))
df.info()

3835 3850
<class 'pandas.core.frame.DataFrame'>
Index: 3850 entries, 2006-06-15 to 2022-07-26
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Open      3850 non-null   int64  
 1   High      3850 non-null   int64  
 2   Low       3850 non-null   int64  
 3   Close     3850 non-null   int64  
 4   Volume    3850 non-null   int64  
 5   Change    3850 non-null   float64
 6   C-Close   3850 non-null   float64
 7   C-Change  3850 non-null   float64
 8   I-Close   3850 non-null   float64
 9   I-Change  3850 non-null   float64
dtypes: float64(5), int64(5)
memory usage: 330.9+ KB


In [9]:
prog_bar = notebook.tqdm(data_files)

for data_file in prog_bar:
    df = load_csv_data(data_file)
    new_df_path = data_file.replace(
        os.path.dirname(data_files[0]), os.path.dirname(new_data_files[0])
    )
    if os.path.exists(new_df_path):
        df_new = load_csv_data(new_df_path)
        df_ = pd.concat([df, df_new])
        df = df_.drop_duplicates().copy()
#         print(f'new data {idx} added')
    
    idx = data_file.split('_')[1]
    prog_bar.set_description_str(idx)
    
    high, low, close, volume = df['High'], df['Low'], df['Close'], df['Volume']
    
    for indicator in INDICATORS:
        data = get_indicator(indicator, high=high, low=low, close=close, volume=volume)
        df[indicator] = data

    df = df[119:]  # drop ma_120 nan
    scaled_df = scale_data(df, idx)
    if scaled_df is None:
        continue
    
    df.to_csv(os.path.join(DATA_DIR, f'original_data_{idx}' + DATA_EXT))
    scaled_df.to_csv(os.path.join(SCALED_DATA_DIR, f'scaled_data_{idx}' + DATA_EXT))